In [ ]:
#Import function library path
from IPython.display import display
from uuid import uuid1
import inspect
import ctypes
import traitlets
import os
import ipywidgets.widgets as widgets
import PID
import traitlets
from jetbot import *
import threading
import time
import cv2
import traitlets
import numpy as np
#end import function library
# is red obj pres or not. 1 when red object present else 0
status = 0 
# lower and upper bound of BGR values the robit will detect
global color_lower
global color_upper
# xy coordinate of center of detected color and radius of detected color
global color_x, color_y, color_radius

camera = Camera.instance(width=300, height=300)
#image = widgets.Image(format='jpeg', width=600, height=224)  # this width and height doesn't necessarily have to match the camera

robot = Robot()

color_image = widgets.Image(format='jpeg', width=600, height=300)
camera_link = traitlets.dlink((camera, 'value'), (color_image, 'value'), transform=bgr8_to_jpeg)

color_x = color_y = color_radius = 0

display(color_image)

# assign values to color bounds. Current value assigned 2/10/23 is [0,0,100] __> [75,75,255] which covers area of red
color_lower=np.array([0,0,100])
color_upper = np.array([75, 75, 255])

def jetbot_motion_center_x():
    while 1:
        if status == 0:
            #no red object then the robot does not move
            robot.stop()
            time.sleep(.01)
        else:
            #red object detected the robot move
            if color_x >= 170:
                #red object detected on right of camera rotate right
                setleft = .75
                setright = -.75 
            elif color_x <= 130:
                #red object detected on left of camera rotate left
                setleft = -.75
                setright = .75
            else :
                #red object on camera center move straight
                setleft = 1
                setright = 1
                
            robot.set_motors(setleft,setright)
            time.sleep(.01)

thread2 = threading.Thread(target=jetbot_motion_center_x)
thread2.setDaemon(True)
thread2.start()

while 1:
    #capture camera frame
    frame = camera.value
    #resize camera frame idk y
    frame = cv2.resize(frame, (300, 300))
    #add gaussian blur. this has yet to improve performace but it came stock
    frame_=cv2.GaussianBlur(frame,(1,1),0)  
    mask=cv2.inRange(frame,color_lower,color_upper)
    mask=cv2.erode(mask,None,iterations=2)
    mask=cv2.dilate(mask,None,iterations=2)
    mask=cv2.GaussianBlur(mask,(1,1),0)     
    cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    #display x and y coordinate. only used for understanding and debugging
    """print("coor x: " + str(color_x) + "\tcoor y: " + str(color_y))"""
    if len(cnts)>0:
        cnt = max (cnts,key=cv2.contourArea)
        (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
        if color_radius > 2:
            # Mark the detected color with circle
            #left is x=0; right is x = 255
            #high is y=0; low is y = 255
            status = 1
            cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  #might uncommment
            #end mark detected color with circle
            # Mark the detected color with rectangle
            """rot_rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rot_rect)
            box = np.int0(box)
            cv2.drawContours(frame,[box],0,(0,0,0,),2)"""
            #end mark detected color with rectangle
    else:
        status = 0
    # Real-time return of image data for display
    color_image.value = bgr8_to_jpeg(frame)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-dfac873e7dc5>", line 46, in jetbot_motion_center_x
    robot.stop()
AttributeError: module 'jetbot.robot' has no attribute 'stop'



Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…